In [44]:
using LinearAlgebra
using SparseArrays
using Plots
include("arnoldi.jl")

arnoldi (generic function with 1 method)

In [45]:
n = 100
M = 30
L = 4
#^^^^^^^^^^^^^^^^^^^^^^^#
#  A*X_σ - X_σ * D = B  #
#_______________________#
# LHS
#   A
λ = [i+10 for i in 1:n]
a = randn(n, n)
A = UpperTriangular(a) - diagm(diag(a)) + diagm(λ)
#   Shift
σ = rand(1:9, L)
D = diagm(σ) 
# RHS
B = randn(n, L);

In [46]:
# Initialize
X_σ     = zeros(n, L) 
X_exact = zeros(n, L)
Res     = zeros(M, L);

In [47]:
# [A](nxn) x = b
# [Q](n x m+1) = [q1 q2 ... q(m+1)], q1 = normalized b
# [H](m+1 x m): upper Hessenberg
# A Q_m = Q H , [Q_m](nxm) = [q1 q2 ... qm]
# x = Q_m z
# A Q_m z = b = Q H z
# Q' b = Q' Q H z => H z = Q' b = s => z = H \ s => x = Q_m z
for l=1:L
    lhs = A - D[l, l]*I
    b = B[:, l]
    X_exact[:, l] = lhs \ b
    Q, H = arnoldi(lhs, b, M)
    @show norm(lhs*Q[:, 1:M] - Q*H)
    for m in 1:M
        s = vcat(norm(b), zeros(m, 1)) # s = Q' b
        z = H[1:m+1, 1:m] \ s          # z = H \ s
        x = Q[:, 1:m] * z              # x = Q_m z
        # Populate results
        Res[m, l] = norm(b - lhs*x)
        X_σ[:, l] = x
    end
end
norm(X_exact - X_σ)

norm(lhs * Q[:, 1:M] - Q * H) = 1.024259293270996e-13
norm(lhs * Q[:, 1:M] - Q * H) = 1.0939229429435777e-13
norm(lhs * Q[:, 1:M] - Q * H) = 1.1153827680532373e-13
norm(lhs * Q[:, 1:M] - Q * H) = 1.0963209394621552e-13


5.557158435166617e-6